# Analisis de cuentas de un banco ficticio para saber si los clientes van a abandonar el banco o no

## Part 1: Data Preprocessing

### Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


### Importing datasets
Las primeras 3 columnas son numero de columna, id de cliente, y nombre de cliente, por lo que no nos interesan.
La ultima columna dice si dejó o no dejó el banco después de 6 meses

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

### Encoding the categorical variables
Enumera el nombre del país y el genero, y no me acuerdo que hace el hot encoder

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:,2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
X.shape

(10000, 11)

### Splitting the dataset into the Training set and Test set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape

(8000, 11)

### Feature scaling
ACP

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train.shape

(8000, 11)

## Part 2: Now let's make the ANN!

### Importing the Keras libraries and packages

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Initializing the ANN

In [7]:
classifier = Sequential()

### Adding the input layer and the first hidden layer
6 neuronas, inicializa los pesos con una distribucion uniforme, relus, 12 entradas

In [8]:
classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

### Adding the second hidden layer
6 neuronas, uniforme, relu

In [9]:
classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu'))

### Adding the output layer
1 neurona, uniforme, binaria

In [10]:
classifier.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))

### Compiling the ANN
b-prop

In [11]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fitting the ANN to the Training set

In [43]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 3s 380us/step - loss: 0.4892 - acc: 0.7955
Epoch 2/100
8000/8000 [==============================] - 2s 293us/step - loss: 0.4295 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 294us/step - loss: 0.4247 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 293us/step - loss: 0.4204 - acc: 0.8075
Epoch 5/100
8000/8000 [==============================] - 2s 299us/step - loss: 0.4173 - acc: 0.8247
Epoch 6/100
8000/8000 [==============================] - 2s 294us/step - loss: 0.4150 - acc: 0.8285
Epoch 7/100
8000/8000 [==============================] - 2s 297us/step - loss: 0.4138 - acc: 0.8292
Epoch 8/100
8000/8000 [==============================] - 2s 297us/step - loss: 0.4120 - acc: 0.8319
Epoch 9/100
8000/8000 [==============================] - 2s 304us/step - loss: 0.4106 - acc: 0.8327
Epoch 10/100
8000/8000 [==============================] - 2s 303us/step - loss: 0.4101 - acc: 0.8339

## Part 3: Making the predictions and evaluating the model

### Predicting the Test set results

In [51]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5) # tupla de booleanos

### Making the Confusion Matrix

In [52]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [53]:
cm

array([[1545,   50],
       [ 263,  142]])

### Predicting a single new observation
credit score: 600  
gender: male  
geography: france  
age: 40  
tenure: 3  
balance: 60000  
number of products: 2  
has credit card: yes  
is active member: yes  
estimated salary: 50000

In [79]:
homework_client = np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])
homework_client = sc.transform(homework_client)
new_prediction = classifier.predict(homework_client)
print(new_prediction)

[[ 0.14901029]]


/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Part 4: Evaluating, Improving and Tunning the ANN
### Evaluating
Keras wrapper para sklearn y una funcion que crea la arquitectura de la red neuronal

In [13]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(
    build_fn = build_classifier, 
    batch_size=10, 
    epochs=100)
accuracies = cross_val_score(
    estimator = classifier,
    X = X_train,
    y = y_train,
    cv = 10,
    n_jobs = -1)

Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 5s 734us/step - loss: 0.4896 - acc: 0.7962
Epoch 2/100
7200/7200 [==============================] - 6s 769us/step - loss: 0.4898 - acc: 0.7950
Epoch 2/100
7200/7200 [==============================] - 4s 572us/step - loss: 0.4295 - acc: 0.7967
Epoch 3/100
7200/7200 [==============================] - 4s 571us/step - loss: 0.4316 - acc: 0.7956
Epoch 3/100
7200/7200 [==============================] - 4s 574us/step - loss: 0.4246 - acc: 0.7967
Epoch 4/100
7200/7200 [==============================] - 4s 547us/step - loss: 0.4231 - acc: 0.8024
Epoch 5/100
7200/7200 [==============================] - 4s 537us/step - loss: 0.4149 - acc: 0.8285
Epoch 5/100
7200/7200 [==============================] - 4s 544us/step - loss: 0.4202 - acc: 0.8108
Epoch 5/100
7200/7200 [==============================] - 4s 532us/step - loss: 0.4177 - acc: 0.8224
Epoch 6/100
7200/7200 [==============================] - 4s 563u

7200/7200 [==============================] - 4s 503us/step - loss: 0.3939 - acc: 0.8362
Epoch 18/100
7200/7200 [==============================] - 4s 513us/step - loss: 0.4067 - acc: 0.8357
Epoch 18/100
7200/7200 [==============================] - 4s 506us/step - loss: 0.4014 - acc: 0.8344
Epoch 18/100
7200/7200 [==============================] - 3s 431us/step - loss: 0.3935 - acc: 0.8374
Epoch 19/100
7200/7200 [==============================] - 3s 419us/step - loss: 0.4011 - acc: 0.8342
Epoch 19/100
7200/7200 [==============================] - 3s 350us/step - loss: 0.4064 - acc: 0.8354
Epoch 20/100
7200/7200 [==============================] - 3s 371us/step - loss: 0.4053 - acc: 0.8369
Epoch 20/100
7200/7200 [==============================] - 3s 374us/step - loss: 0.3931 - acc: 0.8368
Epoch 21/100
7200/7200 [==============================] - 3s 395us/step - loss: 0.4044 - acc: 0.8347
Epoch 21/100
7200/7200 [==============================] - 3s 422us/step - loss: 0.3932 - acc: 0.8364
Epo

7200/7200 [==============================] - 3s 356us/step - loss: 0.4027 - acc: 0.8357
Epoch 34/100
7200/7200 [==============================] - 3s 380us/step - loss: 0.4022 - acc: 0.8368
Epoch 34/100
7200/7200 [==============================] - 3s 401us/step - loss: 0.3914 - acc: 0.8372
Epoch 35/100
7200/7200 [==============================] - 3s 381us/step - loss: 0.3958 - acc: 0.8378
Epoch 35/100
7200/7200 [==============================] - 3s 396us/step - loss: 0.4026 - acc: 0.8354
Epoch 35/100
7200/7200 [==============================] - 3s 360us/step - loss: 0.4017 - acc: 0.8343
Epoch 35/100
7200/7200 [==============================] - 3s 356us/step - loss: 0.3910 - acc: 0.8381
Epoch 36/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3955 - acc: 0.8376
Epoch 36/100
7200/7200 [==============================] - 3s 377us/step - loss: 0.4026 - acc: 0.8354
Epoch 36/100
7200/7200 [==============================] - 3s 363us/step - loss: 0.3912 - acc: 0.8374
Epo

7200/7200 [==============================] - 3s 373us/step - loss: 0.4001 - acc: 0.8353
Epoch 50/100
7200/7200 [==============================] - 3s 368us/step - loss: 0.3903 - acc: 0.8372
Epoch 51/100
7200/7200 [==============================] - 3s 361us/step - loss: 0.3942 - acc: 0.8396
Epoch 51/100
7200/7200 [==============================] - 3s 357us/step - loss: 0.4012 - acc: 0.8367
Epoch 51/100
7200/7200 [==============================] - 3s 369us/step - loss: 0.3997 - acc: 0.8374
Epoch 51/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3901 - acc: 0.8379
Epoch 52/100
7200/7200 [==============================] - 3s 391us/step - loss: 0.3940 - acc: 0.8375
Epoch 52/100
7200/7200 [==============================] - 3s 374us/step - loss: 0.4014 - acc: 0.8360
Epoch 52/100
7200/7200 [==============================] - 3s 367us/step - loss: 0.3998 - acc: 0.8360
Epoch 52/100
7200/7200 [==============================] - 3s 367us/step - loss: 0.3892 - acc: 0.8382
Epo

7200/7200 [==============================] - 3s 372us/step - loss: 0.3899 - acc: 0.8383
Epoch 67/100
7200/7200 [==============================] - 3s 421us/step - loss: 0.3963 - acc: 0.8375
Epoch 67/100
7200/7200 [==============================] - 3s 419us/step - loss: 0.3930 - acc: 0.8375
Epoch 68/100
7200/7200 [==============================] - 3s 386us/step - loss: 0.4008 - acc: 0.8358
Epoch 68/100
7200/7200 [==============================] - 3s 376us/step - loss: 0.3904 - acc: 0.8367
Epoch 69/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3935 - acc: 0.8385
Epoch 69/100
7200/7200 [==============================] - 3s 367us/step - loss: 0.3963 - acc: 0.8360
Epoch 69/100
7200/7200 [==============================] - 3s 369us/step - loss: 0.4011 - acc: 0.8353
Epoch 69/100
7200/7200 [==============================] - 3s 364us/step - loss: 0.3900 - acc: 0.8374
Epoch 70/100
7200/7200 [==============================] - 3s 354us/step - loss: 0.3934 - acc: 0.8369
Epo

...

7200/7200 [==============================] - 3s 365us/step - loss: 0.3407 - acc: 0.8619
Epoch 65/100
7200/7200 [==============================] - 3s 381us/step - loss: 0.3392 - acc: 0.8604
Epoch 66/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3400 - acc: 0.8601
Epoch 66/100
7200/7200 [==============================] - 3s 401us/step - loss: 0.3407 - acc: 0.8600
Epoch 66/100
7200/7200 [==============================] - 3s 359us/step - loss: 0.3436 - acc: 0.8612
Epoch 67/100
7200/7200 [==============================] - 3s 355us/step - loss: 0.3400 - acc: 0.8622
Epoch 67/100
7200/7200 [==============================] - 3s 367us/step - loss: 0.3401 - acc: 0.8606
Epoch 67/100
7200/7200 [==============================] - 3s 348us/step - loss: 0.3395 - acc: 0.8608
Epoch 68/100
7200/7200 [==============================] - 3s 360us/step - loss: 0.3439 - acc: 0.8593
Epoch 68/100
7200/7200 [==============================] - 3s 380us/step - loss: 0.3406 - acc: 0.8611
Epo

7200/7200 [==============================] - 3s 369us/step - loss: 0.3390 - acc: 0.8619
Epoch 82/100
7200/7200 [==============================] - 3s 383us/step - loss: 0.3392 - acc: 0.8610
Epoch 82/100
7200/7200 [==============================] - 3s 381us/step - loss: 0.3416 - acc: 0.8611
Epoch 83/100
7200/7200 [==============================] - 3s 376us/step - loss: 0.3393 - acc: 0.8603
Epoch 83/100
7200/7200 [==============================] - 3s 378us/step - loss: 0.3401 - acc: 0.8612
Epoch 83/100
7200/7200 [==============================] - 3s 374us/step - loss: 0.3377 - acc: 0.8592
Epoch 84/100
7200/7200 [==============================] - 3s 392us/step - loss: 0.3414 - acc: 0.8594
Epoch 84/100
7200/7200 [==============================] - 3s 379us/step - loss: 0.3387 - acc: 0.8594
Epoch 84/100
7200/7200 [==============================] - 3s 365us/step - loss: 0.3371 - acc: 0.8624
Epoch 85/100
7200/7200 [==============================] - 3s 374us/step - loss: 0.3392 - acc: 0.8596
Epo

7200/7200 [==============================] - 2s 300us/step - loss: 0.3394 - acc: 0.8631
Epoch 99/100
7200/7200 [==============================] - 7s 957us/step - loss: 0.3377 - acc: 0.8633
Epoch 99/100
7200/7200 [==============================] - 10s 1ms/step - loss: 0.3400 - acc: 0.8615
Epoch 100/100
7200/7200 [==============================] - 11s 1ms/step - loss: 0.3387 - acc: 0.8611
Epoch 100/100
800/800 [==============================] - 0s 203us/step
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 3s 448us/step - loss: 0.4825 - acc: 0.7957
Epoch 2/100
7200/7200 [==============================] - 3s 443us/step - loss: 0.4832 - acc: 0.7961
Epoch 2/100
7200/7200 [==============================] - 3s 364us/step - loss: 0.4236 - acc: 0.7957
Epoch 3/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.4278 - acc: 0.7961
Epoch 3/100
7200/7200 [==============================] - 3s 361us/step - loss: 0.4179 - acc: 0.8115
Epoch 4/100
7200/7200 [====

Epoch 33/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.3991 - acc: 0.8360
Epoch 33/100
7200/7200 [==============================] - 2s 317us/step - loss: 0.4046 - acc: 0.8339
Epoch 34/100
7200/7200 [==============================] - 2s 295us/step - loss: 0.3987 - acc: 0.8369
Epoch 35/100
7200/7200 [==============================] - 2s 299us/step - loss: 0.4050 - acc: 0.8335
Epoch 35/100
7200/7200 [==============================] - 2s 293us/step - loss: 0.3984 - acc: 0.8365
Epoch 36/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.4049 - acc: 0.8335
Epoch 36/100
7200/7200 [==============================] - 2s 292us/step - loss: 0.3990 - acc: 0.8361
Epoch 37/100
7200/7200 [==============================] - 2s 287us/step - loss: 0.4051 - acc: 0.8337
Epoch 37/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.3986 - acc: 0.8361
Epoch 38/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.4048 - ac

7200/7200 [==============================] - 2s 282us/step - loss: 0.4028 - acc: 0.8333
Epoch 72/100
7200/7200 [==============================] - 2s 294us/step - loss: 0.3972 - acc: 0.8383
Epoch 73/100
7200/7200 [==============================] - 2s 289us/step - loss: 0.4034 - acc: 0.8350
Epoch 73/100
7200/7200 [==============================] - 2s 306us/step - loss: 0.3976 - acc: 0.8371
Epoch 74/100
7200/7200 [==============================] - 2s 305us/step - loss: 0.4034 - acc: 0.8362
Epoch 74/100
7200/7200 [==============================] - 2s 289us/step - loss: 0.4034 - acc: 0.8354
Epoch 75/100
7200/7200 [==============================] - 2s 294us/step - loss: 0.3972 - acc: 0.8383
Epoch 76/100
7200/7200 [==============================] - 2s 282us/step - loss: 0.4033 - acc: 0.8336
Epoch 76/100
7200/7200 [==============================] - 2s 304us/step - loss: 0.3974 - acc: 0.8376
Epoch 77/100
7200/7200 [==============================] - 2s 297us/step - loss: 0.4032 - acc: 0.8344
Epo

In [17]:
print(accuracies.mean())
print(accuracies.std())

0.844249995165
0.0161361391773


### Improving the ANN
### Dropout regularization to reduce overfitting if needed
p entre 0.1 y 0.5

In [18]:
from keras.layers import Dropout
classifier = Sequential()
classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(p = 0.1))
classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(p = 0.1))
classifier.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


### Tunning the ANN

In [ ]:
from sklearn.model_selection import GridSearchCV
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
#grid search
parameters = {
    'batch_size': [25, 32],
    'epochs': [100, 500],
    'optimizer': ['adam', 'rmsprop']
}
grid_search = GridSearchCV(
    estimator = classifier,
    param_grid = parameters,
    scoring = 'accuracy',
    cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters, best_accuracy = grid_search.best_parameters, grid_search.best_accuracy